In [24]:
%run 0.0_Dependance.ipynb

In [25]:
df = pd.read_csv("train.csv",index_col=0)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)

#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

In [26]:
param_grid_initial = {
    # Nombre d'arbres dans le boosting
    'n_estimators': [50, 100, 250, 500, 1000],
    
    # Profondeur de chaque arbre
    'max_depth': [3, 6, 9, 12],
    
    # Taux d'apprentissage (eta)
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    
    # Minimum de perte de réduction nécessaire pour faire une partition supplémentaire sur un noeud de l'arbre
    'gamma': [0, 0.1, 0.5, 1],
    
    # Fraction des observations à être randomisées pour chaque arbre
    'subsample': [0.5, 0.75, 1],
    
    # Fraction de colonnes à être randomisées pour chaque arbre
    'colsample_bytree': [0.5, 0.75, 1],
    
    # Le minimum de poids nécessaire pour une enfant
    'min_child_weight': [1, 5, 10],
    
    # Regularisation L1 sur les poids
    'reg_alpha': [0, 0.1, 1],
    
    # Regularisation L2 sur les poids
    'reg_lambda': [0, 1, 10],
    
    # Choix de la fonction objectif, logloss pour la classification
    'objective': ['multi:softprob'],
    
    # Choix de la stratégie d'évaluation pour le boosting
    'eval_metric': ['mlogloss'],
    
    # Choix du booster
    'booster': ['gbtree'],
}

In [27]:
param_grid_ameliore = {
    # Nombre d'arbres dans le boosting
    'n_estimators': [400,500,600,700,800,900],
    
    # Profondeur de chaque arbre
    'max_depth': [4,5,6],
    
    # Taux d'apprentissage (eta)
    'learning_rate': [0.05,0.06,0.07,0.08,0.09,0.1],
    
    # Minimum de perte de réduction nécessaire pour faire une partition supplémentaire sur un noeud de l'arbre
    'gamma': [0.5,0.7, 1],
    
    # Fraction des observations à être randomisées pour chaque arbre
    'subsample': [0.75, 1],
    
    # Fraction de colonnes à être randomisées pour chaque arbre
    'colsample_bytree': [0.2,0.4,0.5,0.6],
    
    # Le minimum de poids nécessaire pour une enfant
    'min_child_weight': [ 5,6,7,8,9,10],
    
    # Regularisation L1 sur les poids
    'reg_alpha': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1],
    
    # Regularisation L2 sur les poids
    'reg_lambda': [0, 1,2,3,4,5,6,7,8,9,10],
    
    # Choix de la fonction objectif, logloss pour la classification
    'objective': ['multi:softprob'],
    
    # Choix de la stratégie d'évaluation pour le boosting
    'eval_metric': ['mlogloss'],
    
    # Choix du booster
    'booster': ['gbtree'],
}


In [28]:
param_grid_ameliore2 = {
    # Nombre d'arbres dans le boosting
    'n_estimators': [300,400,500,600],
    
    # Profondeur de chaque arbre
    'max_depth': [4,5,6],
    
    # Taux d'apprentissage (eta)
    'learning_rate': [0.05],
    
    # Minimum de perte de réduction nécessaire pour faire une partition supplémentaire sur un noeud de l'arbre
    'gamma': [0.5,0.7, 1],
    
    # Fraction des observations à être randomisées pour chaque arbre
    'subsample': [0.75, 1],
    
    # Fraction de colonnes à être randomisées pour chaque arbre
    'colsample_bytree': [0.2],
    
    # Le minimum de poids nécessaire pour une enfant
    'min_child_weight': [5,6,7],
    
    # Regularisation L1 sur les poids
    'reg_alpha': [0.4,0.5,0.6],
    
    # Regularisation L2 sur les poids
    'reg_lambda': [0, 1,2,3,4,5,6,7,8,9,10],
    
    # Choix de la fonction objectif, logloss pour la classification
    'objective': ['multi:softprob'],
    
    # Choix de la stratégie d'évaluation pour le boosting
    'eval_metric': ['mlogloss'],
    
    # Choix du booster
    'booster': ['gbtree'],
}


In [32]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

joblib_path_suivi_metrique_1 = get_output_path_file("df_score_random_v1.joblib")
joblib_path_suivi_metrique_2 = get_output_path_file("df_score_best_test_v1.joblib")

target = df.Status
data = df.drop(columns=["Status","N_Days"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)
#Standardisation
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#Modèle instancié
model = XGBClassifier(n_estimators=1000, max_depth=2, learning_rate=0.1, objective="multi:softprob")

randomized_search_CV = RandomizedSearchCV(model, param_distributions=param_grid_ameliore2,n_iter=30000, cv=5, scoring='neg_log_loss',n_jobs=-1)
df_score_random_v1_1 = test_randomized_model(joblib_path_suivi_metrique_1, param_grid_ameliore2, model, randomized_search_CV)
dump(df_score_random_v1_1,"df_score_random_v1_1.joblib")

Récupération du DataFrames existant
Début du randomized: RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                   param_distributions={'booster': ['gbtree'],
                  

C:\Users\franc\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 7128 is smaller than n_iter=30000. Running 7128 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fin du randomized: RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                   param_distributions={'booster': ['gbtree'],
                                        'colsample_bytre

['df_score_random_v1_1.joblib']

In [6]:
from joblib import load,dump
dump(df_score_random_v1_1,"df_score_random_v1_1.joblib")

['df_score_random_v1.joblib']

In [33]:
from joblib import load,dump
df_score_random_v1_1 = load("df_score_random_v1_1.joblib")
df_score_random_v1_1 = df_score_random_v1_1.drop_duplicates()
dump(df_score_random_v1_1,"df_score_random_v1_1.joblib")


['df_score_random_v1_1.joblib']

In [35]:
df_score_random_v1_1

,n_estimators,max_depth,learning_rate,gamma,subsample,colsample_bytree,min_child_weight,reg_alpha,reg_lambda,objective,eval_metric,booster,LogLoss_mean,LogLoss_std
0,500,5,0.08,0.7,0.75,0.2,9,0.3,2,multi:softprob,mlogloss,gbtree,-0.430215,0.014266
1,900,4,0.09,0.5,0.75,0.2,10,0.6,1,multi:softprob,mlogloss,gbtree,-0.436219,0.015202
2,900,6,0.08,1,1,0.5,5,0.3,5,multi:softprob,mlogloss,gbtree,-0.442235,0.013131
3,800,6,0.08,0.5,0.75,0.2,7,0.6,9,multi:softprob,mlogloss,gbtree,-0.433717,0.014389
4,900,4,0.05,0.5,1,0.2,6,0.9,3,multi:softprob,mlogloss,gbtree,-0.434770,0.012334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77984,600,6,0.05,1,1,0.2,7,0.6,8,multi:softprob,mlogloss,gbtree,-0.437358,0.012090
77985,600,6,0.05,1,0.75,0.2,7,0.6,9,multi:softprob,mlogloss,gbtree,-0.436323,0.012936
77986,600,6,0.05,1,1,0.2,7,0.6,9,multi:softprob,mlogloss,gbtree,-0.438068,0.012563
77987,600,6,0.05,1,0.75,0.2,7,0.6,10,multi:softprob,mlogloss,gbtree,-0.436671,0.012938


In [22]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
import numpy as np
from joblib import Parallel, delayed
from joblib import dump,load

def load_create_dftop(df_path, n_top):
    df = load(df_path)
    df_sorted = df.sort_values(by="LogLoss_mean", ascending=False)
    df_top = df_sorted.iloc[:n_top, :]
    return df_top

# Fonction pour entraîner le modèle et calculer le log_loss
def compute_log_loss(i, df, X_train, y_train, X_test, y_test):
    dico = df.iloc[i, :-2].to_dict()
    model = XGBClassifier(**dico)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    return log_loss(y_test, probs)

df_score_random_v1_top_1000 = load_create_dftop("df_score_random_v1.joblib", 1000)

# Parallélisation du calcul du log_loss
log_losses = Parallel(n_jobs=-1)(delayed(compute_log_loss)(i, df_score_random_v1_top_1000, X_train, y_train, X_test, y_test) for i in range(1000))

df_score_random_v1_top_1000['Test_log_loss'] = log_losses
df_score_random_v1_top_1000 = df_score_random_v1_top_1000.sort_values(by="Test_log_loss",ascending=True)
dump(df_score_random_v1_top_1000,"df_score_random_v1_top_1000.joblib")

['df_score_random_v1_top_1000.joblib']

In [23]:
df_score_random_v1_top_1000

,n_estimators,max_depth,learning_rate,gamma,subsample,colsample_bytree,min_child_weight,reg_alpha,reg_lambda,objective,eval_metric,booster,LogLoss_mean,LogLoss_std,Test_log_loss
54802,400,6,0.05,0.5,1,0.2,6,0.3,1,multi:softprob,mlogloss,gbtree,-0.427643,0.012277,0.438500
69650,400,6,0.05,0.5,0.75,0.2,10,0.2,1,multi:softprob,mlogloss,gbtree,-0.429360,0.012945,0.438810
40952,600,6,0.05,0.5,1,0.2,5,0.2,6,multi:softprob,mlogloss,gbtree,-0.428350,0.012862,0.439039
34044,400,6,0.05,0.5,1,0.2,5,0.5,1,multi:softprob,mlogloss,gbtree,-0.427807,0.012810,0.439241
53891,400,6,0.05,0.5,1,0.2,6,0.1,2,multi:softprob,mlogloss,gbtree,-0.427414,0.012488,0.439437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17381,500,5,0.1,0.5,0.75,0.2,5,0.9,1,multi:softprob,mlogloss,gbtree,-0.429741,0.014865,0.451678
38919,500,6,0.09,0.5,0.75,0.2,5,0.3,3,multi:softprob,mlogloss,gbtree,-0.429689,0.014921,0.452398
10831,800,6,0.08,0.7,0.75,0.2,6,0.1,0,multi:softprob,mlogloss,gbtree,-0.429614,0.014190,0.452574
48709,600,5,0.09,0.7,0.75,0.2,6,0.2,0,multi:softprob,mlogloss,gbtree,-0.429740,0.014534,0.453049
